---

Created for [Pricing and Hedging Derivative Securities: Theory and Methods](https://book.derivative-securities.org/)

Authored by
- Kerry Back, Rice University
- Hong Liu, Washington University in St. Louis
- Mark Loewenstein, University of Maryland
 
---

<a target="_blank" href="https://colab.research.google.com/github/math-finance-book/book-code/blob/main/19_Jumps.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<img src="https://www.dropbox.com/scl/fi/6hwvdff7ajaafmkpmnp0o/under_construction.jpg?rlkey=3dex2dx86anniqoutwyqashnu&dl=1" alt="Under Construction" width="500"/>


In [ ]:



import numpy as np
import matplotlib.pyplot as plt

def simulate_jump_diffusion(S0, mu, sigma, lamb, m, delta, T, N):
    """
    Simulate a jump diffusion process.
    
    Parameters:
    S0     : float - initial stock price
    mu     : float - drift rate
    sigma  : float - volatility
    lamb   : float - intensity of the Poisson process
    m      : float - mean of the jump size distribution
    delta  : float - standard deviation of the jump size distribution
    T      : float - total time
    N      : int   - number of time steps
    
    Returns:
    t      : numpy array - time points
    S      : numpy array - simulated stock prices
    """
    dt = T / N
    t = np.linspace(0, T, N + 1)
    S = np.zeros(N + 1)
    S[0] = S0
    
    for i in range(1, N + 1):
        Z = np.random.normal(0, 1)  # Normal random variable for the diffusion part
        J = np.random.poisson(lamb * dt)  # Poisson random variable for jumps
        
        # Sum of log-normal distributed jumps
        Y = np.sum(np.random.normal(m, delta, J))
        
        # Update stock price
        S[i] = S[i - 1] * np.exp((mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z + Y)
    
    return t, S

# Parameters
S0 = 100         # Initial stock price
mu = 0.1         # Drift rate
sigma = 0.2      # Volatility
lamb = 0.75      # Intensity of the Poisson process (average number of jumps per unit time)
m = 0.02         # Mean of the jump size distribution (log-normal)
delta = 0.1      # Standard deviation of the jump size distribution (log-normal)
T = 1.0          # Total time (1 year)
N = 1000         # Number of time steps

# Simulate the jump diffusion process
t, S = simulate_jump_diffusion(S0, mu, sigma, lamb, m, delta, T, N)

# Plot the simulated stock prices
plt.figure(figsize=(10, 6))
plt.plot(t, S, label='Jump Diffusion Process')
plt.title('Simulated Stock Price using Jump Diffusion Model')
plt.xlabel('Time (years)')
plt.ylabel('Stock Price')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import brentq

# Heston model parameters
S0 = 100      # Initial stock price
V0 = 0.04     # Initial variance
r = 0.05      # Risk-free rate
q = 0.01      # Dividend yield
T = 1         # Time to maturity (in years)
kappa = 0.25   # Rate of mean reversion of variance
theta = 0.04  # Long-term variance
sigma = 0.5   # Volatility of variance
rho = -0.2    # Correlation between the two Wiener processes
dt = 1/252    # Length of each time period (daily)
N = 252       # Number of time periods (one year)
n_simulations = 10000  # Number of simulations

def simulate_heston_paths(S0, V0, r, q, T, kappa, theta, sigma, rho, dt, N, n_simulations):
    S = np.zeros((N + 1, n_simulations))
    V = np.zeros((N + 1, n_simulations))
    S[0] = S0
    V[0] = V0
    
    for t in range(1, N + 1):
        Z1 = np.random.normal(size=n_simulations)
        Z2 = np.random.normal(size=n_simulations)
        W1 = Z1
        W2 = rho * Z1 + np.sqrt(1 - rho**2) * Z2
        
        V[t] = np.maximum(V[t-1] + kappa * (theta - V[t-1]) * dt + sigma * np.sqrt(V[t-1] * dt) * W2, 0)
        S[t] = S[t-1] * np.exp((r - q - 0.5 * V[t-1]) * dt + np.sqrt(V[t-1] * dt) * W1)
    
    return S, V

def black_scholes_call_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def implied_volatility(C, S, K, T, r):
    def objective_function(sigma):
        return black_scholes_call_price(S, K, T, r, sigma) - C
    try:
        return brentq(objective_function, 0.001, 5.0)
    except ValueError:
        return np.nan

# Simulate paths using the Heston model
S_paths, _ = simulate_heston_paths(S0, V0, r, q, T, kappa, theta, sigma, rho, dt, N, n_simulations)

# Calculate option prices at different strike prices
strike_prices = np.linspace(90, 120, 20)
call_prices = np.zeros_like(strike_prices)

for i, K in enumerate(strike_prices):
    call_payoffs = np.maximum(S_paths[-1] - K, 0)
    call_prices[i] = np.mean(call_payoffs) * np.exp(-r * T)

# Calculate implied volatilities
implied_vols = [implied_volatility(C, S0, K, T, r) for C, K in zip(call_prices, strike_prices)]

# Filter out NaN values that may occur
valid_indices = ~np.isnan(implied_vols)
strike_prices = strike_prices[valid_indices]
implied_vols = np.array(implied_vols)[valid_indices]

# Plot the implied volatility smile
plt.figure(figsize=(10, 6))
plt.plot(strike_prices, implied_vols, label='Implied Volatility', marker='o')
plt.title('Implied Volatility Smile')
plt.xlabel('Strike Price')
plt.ylabel('Implied Volatility')
plt.legend()
plt.grid(True)
plt.show()